# Functionalization

In [3]:
import torch
import torch_xla
import os

os.environ["XLA_DISABLE_FUNCTIONALIZATION"] = "1"

t = torch.randn(10,10).to('xla')
print(torch_xla._XLAC._get_xla_tensor_debug_info(t))
t += 1
print(torch_xla._XLAC._get_xla_tensor_debug_info(t))
print(print(torch_xla._XLAC._get_xla_tensors_hlo([t])))

XLATensor {
TensorID: 4
AliasID: 4
Device: TPU:0
XLA Shape: f32[10,10]
ShardingSpec: None
IR: [UNKNOWN_SCALAR[]] xla::device_data, xla_shape=f32[10,10]{1,0}, dynamic_dims: (), device=TPU:0
XLAData: None
Tensor on host: None
}

XLATensor {
TensorID: 4
AliasID: 4
Device: TPU:0
XLA Shape: f32[10,10]
ShardingSpec: None
IR: [] aten::add, xla_shape=f32[10,10]{1,0}, dynamic_dims: ()
XLAData: None
Tensor on host: None
}

HloModule IrToHlo.8, entry_computation_layout={(f32[10,10]{1,0})->(f32[10,10]{1,0})}

ENTRY %IrToHlo.8 (p0.3: f32[10,10]) -> (f32[10,10]) {
  %p0.3 = f32[10,10]{1,0} parameter(0)
  %constant.2 = f32[] constant(1)
  %constant.1 = f32[] constant(1)
  %multiply.4 = f32[] multiply(f32[] %constant.2, f32[] %constant.1)
  %broadcast.5 = f32[10,10]{1,0} broadcast(f32[] %multiply.4), dimensions={}
  %add.6 = f32[10,10]{1,0} add(f32[10,10]{1,0} %p0.3, f32[10,10]{1,0} %broadcast.5)
  ROOT %tuple.7 = (f32[10,10]{1,0}) tuple(f32[10,10]{1,0} %add.6)
}


None


In [4]:
os.environ["XLA_DISABLE_FUNCTIONALIZATION"] = "0"

t1 = torch.randn(10,10).to('xla')
print(torch_xla._XLAC._get_xla_tensor_debug_info(t1))
t1 += 1
print(torch_xla._XLAC._get_xla_tensor_debug_info(t1))
print(print(torch_xla._XLAC._get_xla_tensors_hlo([t1])))

XLATensor {
TensorID: 7
AliasID: 7
Device: TPU:0
XLA Shape: f32[10,10]
ShardingSpec: None
IR: [UNKNOWN_SCALAR[]] xla::device_data, xla_shape=f32[10,10]{1,0}, dynamic_dims: (), device=TPU:0
XLAData: None
Tensor on host: None
}

XLATensor {
TensorID: 9
AliasID: 7
Device: TPU:0
XLA Shape: f32[10,10]
ShardingSpec: None
IR: [] aten::add, xla_shape=f32[10,10]{1,0}, dynamic_dims: ()
XLAData: None
Tensor on host: None
}

HloModule IrToHlo.8, entry_computation_layout={(f32[10,10]{1,0})->(f32[10,10]{1,0})}

ENTRY %IrToHlo.8 (p0.3: f32[10,10]) -> (f32[10,10]) {
  %p0.3 = f32[10,10]{1,0} parameter(0)
  %constant.2 = f32[] constant(1)
  %constant.1 = f32[] constant(1)
  %multiply.4 = f32[] multiply(f32[] %constant.2, f32[] %constant.1)
  %broadcast.5 = f32[10,10]{1,0} broadcast(f32[] %multiply.4), dimensions={}
  %add.6 = f32[10,10]{1,0} add(f32[10,10]{1,0} %p0.3, f32[10,10]{1,0} %broadcast.5)
  ROOT %tuple.7 = (f32[10,10]{1,0}) tuple(f32[10,10]{1,0} %add.6)
}


None


`SyncTensorCollection`

Stores the indices of tensors to be synced during mark_step

Includes a hash for the computation that includes the following values:
* config.force_ltc_data: When we materialize the tensor, we trim the IR for all the nodes above them
* HashCompilationEnv: Includes [topology](https://github.com/pytorch/xla/blob/2e4f073abb88ad9b10737036304b3a2bee7d4d4f/torch_xla/csrc/runtime/pjrt_computation_client.cc#L72). If XLA fails to describes topology, this is manually computed.
* XLA_GIT_REV
* 

`GetBufferDonor`

In [ ]:
XlaTensorPtr: Pointer to XlaTensor that is a subclass of LazyTensor.

* GetCurrentDataHandle: Returns BackendDataPtr. This seems like a pointer to actual data on the device but not sure. How to verify this

Checkout [TensorBody.h](https://github.com/pytorch/pytorch/blob/main/aten/src/ATen/templates/TensorBody.h) for aten::Tensor implementation and see how the tensor classes are layered. 

How is the backend data actually stored?
